In [1]:
import cv2
import matplotlib.pyplot as plt
from matplotlib import image
from matplotlib.patches import Circle, Rectangle
import numpy as np
import os
import re
from scipy import spatial
from scipy import signal
from scipy import ndimage
from scipy import optimize

%matplotlib

Using matplotlib backend: Qt5Agg


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


## Creating Photogrammetry Targets

In [2]:
def gen_template(w: float, h: float, dpi: int, loc: str, plot=False):
    '''
    Generate an image of a photogrammetry target.

    Parameters
    ----------
    w
        total image width, m
    h
        total image height, m
    dpi
        output image dpi, converts between matrix coordinates and real width
    loc
        [SW, NW, SE, NE, raft] generates a different pattern for each loc
    '''
    np.random.seed(77777) # generate same noise pattern every time

    IN_TO_METERS = 0.0254
    w_in = w / IN_TO_METERS
    h_in = h / IN_TO_METERS
    w_px = np.round(w_in * dpi).astype(int)
    h_px = np.round(h_in * dpi).astype(int)

    fig, ax = plt.subplots()
    fig.set_size_inches(w_in, h_in)
    fig.tight_layout()
    fig.subplots_adjust(left=0., right=1., top=1., bottom=0.)
    ax.scatter([0, 0, w_px, w_px], [0, h_px, 0, h_px], color='k')

    # black background
    # ax.add_patch(Rectangle((0, 0), w_px, h_px, color='k'))
    pattern_size = 8
    pattern = np.random.random((pattern_size,pattern_size)) * 0.5 + .5
    pattern[0][0] = 0
    pattern[pattern_size - 1][pattern_size - 1] = 1
    ax.imshow(ndimage.zoom(pattern, h_px / pattern_size, order=0), cmap='Greys')

    # center cross
    c_size = dpi * 0.01 / IN_TO_METERS # 10 cm
    c_thick = dpi * 0.001 / IN_TO_METERS # 1 mm
    ax.add_patch(Rectangle((w_px / 2 - c_size / 2, h_px / 2 - c_thick / 2), c_size, c_thick, color='w')) # horz
    ax.add_patch(Rectangle((w_px / 2 - c_thick / 2, h_px / 2 - c_size / 2), c_thick, c_size, color='w')) # vert

    circ_diam = c_size
    if loc == 'SW':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4, h_px / 4),
        )
    elif loc == 'NW':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4 + circ_diam / 2, 0),
            (w_px / 4, 3 * h_px / 4),
        )
    elif loc == 'NE':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4 + circ_diam / 2, 0),
            (2 * w_px / 4 + circ_diam / 2, 0),
            (3 * w_px / 4, 3 * h_px / 4),
        )
    elif loc == 'SE':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4 + circ_diam / 2, 0),
            (2 * w_px / 4 + circ_diam / 2, 0),
            (3 * w_px / 4 + circ_diam / 2, 0),
            (3 * w_px / 4, h_px / 4),
        )
    elif loc == 'raft':
        locs = (
            (w_px / 4, h_px / 4),
            (w_px / 4, 3 * h_px / 4),
            (3 * w_px / 4, 3 * h_px / 4),
            (3 * w_px / 4, h_px / 4),
        )
    else:
        raise ValueError(f'loc {loc} is not an option')

    for elem in locs:
        ax.add_patch(Circle(elem, circ_diam / 2, color='w', zorder=1))

    ax.set_aspect('equal')
    ax.set_xlim([0, w_px])
    ax.set_ylim([0, h_px])
    ax.set_xticks([])
    ax.set_yticks([])
    fig.savefig(f'{loc}_target.png')
    if not plot:
        plt.close()

In [3]:
gen_locs = ['SW', 'NW', 'NE', 'SE', 'raft']
TARGET_W = 0.04 # 40 mm
TARGET_H = TARGET_W
# for loc in locs:
    # gen_template(TARGET_W, TARGET_H, 300, loc, plot=False)

## Doing Photogrammetry

In [4]:
def measure_pos(filename, fallback_px_per_m, fallback_grid_rot):
    # locs = ['SW', 'NW', 'NE', 'SE', 'raft'] # all targets expected to be in the images
    locs = ['SW', 'raft'] # all targets expected to be in the images

    im = image.imread(filename)

    # First, determine the pixel to meter scale from the image
    def get_grid_xy(n_x, n_y, w, h, x0, y0, rot):
        '''A model of a grid of points'''
        xs = np.linspace(x0, x0+w, num=int(n_x))
        ys = np.linspace(y0, y0+h, num=int(n_y))
        X,Y = np.meshgrid(xs, ys)
        rot *= np.pi / 180.
        Xp = X*np.cos(rot) - Y*np.sin(rot) 
        Yp = X*np.sin(rot) + Y*np.cos(rot)
        pts = np.array(list(zip(X.ravel(), Y.ravel())))
        return pts

    def get_grid_xy_err(p0, holes):
        '''Generate residuals of model - detected dot positions from image'''
        pts = get_grid_xy(*p0)
        err = 0
        for i in range(pts.shape[0]):
            for j in range(pts.shape[1]):
                err += np.sqrt(np.sum((pts[i][j] - holes[i][j]) ** 2.))
        return err

    # Set up a blob detector to find the grid of optical breadboard holes in each image
    # https://longervision.github.io/2017/03/18/ComputerVision/OpenCV/opencv-internal-calibration-circle-grid/
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    blobParams = cv2.SimpleBlobDetector_Params()

    blobParams.blobColor = 0
    blobParams.minThreshold = 10
    blobParams.maxThreshold = 200

    blobParams.filterByArea = True
    blobParams.minArea = 300
    blobParams.maxArea = 900

    blobParams.filterByCircularity = True
    blobParams.minCircularity = 0.5

    blobParams.filterByConvexity = True
    blobParams.minConvexity = 0.87

    blobParams.filterByInertia = True
    blobParams.minInertiaRatio = 0.01

    blobDetector = cv2.SimpleBlobDetector_create(blobParams)
    keypoints = blobDetector.detect(im)
    
    # shoot for a shape that should always be unobscured by the raft
    goal_grid_shape = (15,3)

    im_cv = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY) # to 8-bit grey
    im_with_keypoints = cv2.drawKeypoints(im_cv, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)    # Detect the grid of dot positions
    ret, corners = cv2.findCirclesGrid(
        im_cv,
        goal_grid_shape,
        None,
        flags=cv2.CALIB_CB_SYMMETRIC_GRID,
        blobDetector=blobDetector
    )
    if not ret:
        print('grid finding failed, reverting to last px per m value and grid rotation estimate')
        px_per_m = fallback_px_per_m
        grid_rot = fallback_grid_rot
    else:
        im_with_keypoints = cv2.drawChessboardCorners(im_cv, goal_grid_shape, corners, ret)
        # print(corners, len(corners))
        # fig, ax = plt.subplots()
        # ax.set_aspect('equal')
        # ims = ax.imshow(im_with_keypoints)

        holes = corners[:,0,:]
        # ax.scatter(holes[:,0], holes[:,1])
        # if all grid points found, reshaping should work
        holes = holes.reshape((goal_grid_shape[1], goal_grid_shape[0], 2))
        # print(holes.shape, holes[:,:,:].shape)

        # pair up adjacent holes to calculate an average distance - 
        # theoretically we could/should do this for every pair, as they all have
        # some expected distance from each other, but this is the easiest for now.
        dists = np.array([])
        # calculate euclidean distances between points in each row and col
        # to avoid any image rotation from mixing into the spacing estimate
        # https://stackoverflow.com/a/13592234
        for i in range(holes.shape[0]):
            d = np.diff(holes[i,:,:], axis=0)
            dists = np.concatenate([dists, np.sqrt((d ** 2).sum(axis=1))])
        for j in range(holes.shape[1]):
            d = np.diff(holes[:,j,:], axis=0)
            dists = np.concatenate([dists, np.sqrt((d ** 2).sum(axis=1))])
        # print(dists)

        # inch spacing standard for the optical breadboard
        avg_spacing = np.mean(np.abs(dists))
        px_per_m = avg_spacing / (1. * 0.0254)
        print('calculated px_per_m:', px_per_m)

        # find the average rotation of the found grid via the slope:
        # collapse rows
        # collapsed_x = np.array([])
        # collapsed_y = np.array([])
        # for row in holes:
        #     collapsed_x = np.concatenate([collapsed_x, row[:,0]])
        #     collapsed_y = np.concatenate([collapsed_y, row[:,1] - row[:,1][0]])
        # # print(collapsed_x, collapsed_y)
        # # ax.scatter(collapsed_x, collapsed_y)
        # def model(x, m, b):
        #     return m * x + b
        # p0 = [0, 0]
        # popt, pcov = optimize.curve_fit(model,
        #     collapsed_x,
        #     collapsed_y, 
        #     p0,
        # )
        # slope, intc = popt
        # fit_xs = np.linspace(collapsed_x[0], collapsed_x[-1], num=100) 
        # # ax.plot(fit_xs, model(fit_xs, slope, intc))
        # grid_rot = np.arctan2(slope, 1.)
        grid_rot = 0
        # print('calculated image rotation:', 180. * grid_rot / np.pi)

    # Create the main figure to host the ground truth image, xcorr, and template
    f0 = plt.figure(constrained_layout=True)
    f0.suptitle(filename)
    axd = f0.subplot_mosaic(
        """
        ac
        ab
        """
    )
    # Set up truth image for cross-correlation to find target positions
    im_corr = np.sum(im.astype(float), axis=2) # single channel
    im_corr = np.max(im_corr) - im_corr # invert value
    im_corr /= im_corr.max() # normalize
    im_corr -= im_corr.mean() # detrend
    stride = 1 # downsample for speed
    # apply a rotation for both the file's intrinsic rotation (90 deg)
    # and the derived rotation from the detected grid points
    im_corr = ndimage.rotate(im_corr, 90 + (180. * grid_rot / np.pi))
    im_corr = im_corr[::stride,::stride]
    axd['a'].imshow(im_corr, cmap='Greys')
    axd['b'].set_title('Similarity Metric (xcorr)')
    axd['c'].set_title(f'Target Template Image')

    found_pos = []
    psrs = []
    for i, loc in enumerate(locs):
        tmp = image.imread(loc + '_target.png')
        tmp = np.sum(tmp.astype(float) , axis=2) # single channel
        tmp = np.max(tmp) - tmp # invert
        tmp /= tmp.max() # normalize
        tmp -= tmp.mean() # detrend
        
        # Resize the template to have the correct pixel scale in image coordinates
        # targets are assumed square
        tgt_px = TARGET_W * px_per_m
        zoom = tgt_px / tmp.shape[0] / stride
        tmp_zoom = ndimage.zoom(tmp, zoom)
        # find best matching lags in image space
        xcorr = signal.fftconvolve(im_corr, tmp_zoom[::-1, ::-1], mode='valid')
        # a metric for strength of the match
        sidelobe = np.partition(xcorr.flatten(), -2)[-2] # https://stackoverflow.com/q/33181350
        psr = np.max(xcorr) / sidelobe
        psrs.append(psr)
        print(f'{filename} {loc} xcorr peak-to-sidelobe ratio: {psr}')
        
        axd['c'].imshow(tmp_zoom, cmap='Greys')
        axd['b'].imshow(xcorr)

        # the indices of the best match lags
        y,x = np.unravel_index(np.argmax(xcorr), xcorr.shape)
        x_cent = x + tmp_zoom.shape[0] / 2
        y_cent = y + tmp_zoom.shape[1] / 2
        print(f'{loc} target center found at {(x_cent * stride, y_cent * stride)} px in image')
        axd['a'].add_patch(Rectangle((x, y), tmp_zoom.shape[0], tmp_zoom.shape[1], fill=False, color='red'))
        axd['a'].scatter(x_cent, y_cent)
        axd['a'].annotate(f'{loc}', (x,y))
        found_pos.append((x_cent * stride, y_cent * stride))
                
        if loc == 'raft':
            # Go on to try to find the best rotation
            im_crop = im_corr[y:y + tmp_zoom.shape[1], x:x + tmp_zoom.shape[0]]
            rot_psrs = []
            angs = np.arange(-10, 10, .1)
            for ang in angs:
                im_crop_rot = ndimage.rotate(im_crop, ang, reshape=False)
                rot_xcorr = signal.fftconvolve(im_crop_rot, tmp_zoom[::-1, ::-1], mode='valid')
                rot_sidelobe = np.partition(xcorr.flatten(), -2)[-2] # https://stackoverflow.com/q/33181350
                rot_psrs.append(np.max(rot_xcorr) / rot_sidelobe)
            best_rot = angs[np.argmax(rot_psrs)]
            # print('Raft rotation:', best_rot)
            axd['c'].set_title(f'Raft target after derotation by {best_rot:.3f} deg:')
            axd['c'].imshow(ndimage.rotate(im_crop, best_rot, reshape=False))
        
        f0.canvas.draw_idle()
        plt.pause(0.001)

    file_base = os.path.splitext(os.path.basename(filename))[0]
    f0.savefig(f'{file_base}_found_locs.png', dpi=150)
    plt.pause(1)
    plt.close(f0)

    # which position is the raft in?
    sw_loc = np.array(found_pos[0])
    raft_loc = np.array(found_pos[-1])

    return px_per_m, grid_rot, sw_loc, raft_loc, best_rot

In [49]:
def create_files_in_order(dataset_name, xs, ys):
    '''
    Make copies of and rename files according to their position in the error map grid.
    Assumed that photo filenames are ordered such that they depict each position in
    increasing rows of x, increasing columns of y, row-major.

    Should only have to do this if you have taken new photos of the mapper that you wish to analyze.
    '''
    from shutil import copyfile
    import glob

    orig_photos = sorted(glob.glob(os.path.join('img', 'orig', dataset_name, '*.JPG')))
    count = 0
    for y_elem in ys:
        for x_elem in xs:
            # filenames express position offsets from SW in mm
            fname = 'x' + str(np.round(x_elem * 1000, 2)).rstrip('0').rstrip('.') + 'y' + str(np.round(y_elem * 1000, 2)).rstrip('0').rstrip('.') + '.jpeg'
            # print(x_elem, y_elem, orig_photos[count], os.path.join('img', dataset_name, fname))
            copyfile(orig_photos[count], os.path.join('img', dataset_name, fname))
            count += 1

In [50]:
# dataset_name = 'point_to_point'
# xs = np.arange(.15, .55, .05)#[::2] # point_to_point (max coverage scan)
# ys = np.arange(.15, .55, .05)#[::2]
# create_files_in_order(dataset_name, xs, ys)

# dataset_name = '5x5_raster_coarse_K3_v3' # coarse raster scan
# xs = np.linspace(.3, .45, num=5)
# ys = np.linspace(.2, .35, num=5)
# create_files_in_order(dataset_name, xs, ys)

# dataset_name = '5x5_raster_coarse_K3_backstep' # coarse raster scan starting in NW
# xs = np.linspace(.3, .45, num=5)
# ys = np.linspace(.2, .35, num=5)
# flip y-order for file naming, since this scan starts from the top instead of bottom
# create_files_in_order(dataset_name, xs, ys[::-1])

dataset_name = '7x7_raster_24in_breadboard_large_coverage' # coarse raster scan starting in NW
xs = np.linspace(0.19478, 0.41480, num=7) # read out of the input profile .csv file
ys = np.linspace(0.19988, 0.41988, num=7)
# flip y-order for file naming, since this scan starts from the top instead of bottom
create_files_in_order(dataset_name, xs, ys[::-1])

In [51]:
X,Y = np.meshgrid(xs, ys)
meas_sw_pos_px_x = np.zeros(X.shape)
meas_sw_pos_px_y = np.zeros(Y.shape)
meas_raft_pos_px_x = np.zeros(X.shape)
meas_raft_pos_px_y = np.zeros(Y.shape)
x_err = np.zeros(X.shape)
y_err = np.zeros(Y.shape)
raft_ang = np.zeros(Y.shape)
px_per_ms = []
grid_rot_rads = []
fallback_px_per_m = 4560. # magic number from Evan's experience, just in case the first fit is bad.
fallback_grid_rot = (np.pi * .19 / 180.)
for i, x_elem in enumerate(xs):
    for j, y_elem in enumerate(ys):
        # create filename from query points
        fname = 'x' + str(np.round(x_elem * 1000, 2)).rstrip('0').rstrip('.') + 'y' + str(np.round(y_elem * 1000, 2)).rstrip('0').rstrip('.') + '.jpeg'
        print(f'Processing {fname}')
        path = os.path.join('img', dataset_name, fname)
        if os.path.exists(path):
            px_per_m, grid_rot, sw_pos, raft_pos, ang = measure_pos(path, fallback_px_per_m, fallback_grid_rot)
            fallback_px_per_m = px_per_m
            fallback_grid_rot = grid_rot
            px_per_ms.append(px_per_m)
            grid_rot_rads.append(grid_rot)
            meas_sw_pos_px_x[i][j] = sw_pos[0]
            meas_sw_pos_px_y[i][j] = sw_pos[1]
            meas_raft_pos_px_x[i][j] = raft_pos[0]
            meas_raft_pos_px_y[i][j] = raft_pos[1]
            raft_ang[i][j] = ang

Processing x194.78y199.88.jpeg
calculated px_per_m: 3976.5139952732197
img/7x7_raster_24in_breadboard_large_coverage/x194.78y199.88.jpeg SW xcorr peak-to-sidelobe ratio: 1.0097474579061791
SW target center found at (469.5, 3535.5) px in image
img/7x7_raster_24in_breadboard_large_coverage/x194.78y199.88.jpeg raft xcorr peak-to-sidelobe ratio: 1.0164097213678567
raft target center found at (1261.5, 2779.5) px in image
Processing x194.78y236.55.jpeg
grid finding failed, reverting to last px per m value and grid rotation estimate
img/7x7_raster_24in_breadboard_large_coverage/x194.78y236.55.jpeg SW xcorr peak-to-sidelobe ratio: 1.0399656226510567
SW target center found at (469.5, 3529.5) px in image
img/7x7_raster_24in_breadboard_large_coverage/x194.78y236.55.jpeg raft xcorr peak-to-sidelobe ratio: 1.0080572714669065
raft target center found at (1261.5, 2623.5) px in image
Processing x194.78y273.21.jpeg
calculated px_per_m: 4006.3153581460733
img/7x7_raster_24in_breadboard_large_coverage/x1

## Post-Processing and Visualization

In [52]:
# remove outliers due to failed position finding, if necessary (replace with nans)
px_per_ms_clean = px_per_ms.copy()
# print(px_per_ms_clean)
# ax = plt.axes()
# ax.hist(px_per_ms_clean)
# print(np.logspace(6.8, 7.5, num=20, base=2) / TARGET_W)
meas_sw_pos_px_x_clean = meas_sw_pos_px_x.copy()
meas_sw_pos_px_y_clean = meas_sw_pos_px_y.copy()
meas_raft_pos_px_x_clean = meas_raft_pos_px_x.copy()
# meas_raft_pos_px_x_clean[1][0] = np.nan
# meas_raft_pos_px_x_clean[4][0] = np.nan
# meas_raft_pos_px_x_clean[5][6] = np.nan
# meas_raft_pos_px_y_clean[6][4] = np.nan
meas_raft_pos_px_y_clean = meas_raft_pos_px_y.copy()
# meas_raft_pos_px_y_clean[1][0] = np.nan
# meas_raft_pos_px_y_clean[4][0] = np.nan
# meas_raft_pos_px_y_clean[5][6] = np.nan
# meas_raft_pos_px_y_clean[6][4] = np.nan
raft_ang_clean = raft_ang.copy()

In [53]:
# average all best-corr target estimates of pixels per meter to find the image scale
mean_px_per_m = np.nanmean(px_per_ms_clean)
std_px_per_m = np.nanstd(px_per_ms_clean)
f, ax = plt.subplots()
ax.hist(px_per_ms_clean, bins=20)
print('Mean pixels per meter:', mean_px_per_m)
print('Pixel per meter std. dev.:', std_px_per_m)
print('Error in difference of two pixel coordinates:', 2. ** 0.5 * std_px_per_m, 'pixels per meter of distance')
# Convert image coordinates to meters and find raft positions relative to SW in each image
raft_x_mirror_coords = (meas_raft_pos_px_x_clean - meas_sw_pos_px_x_clean) / mean_px_per_m
raft_y_mirror_coords = (meas_sw_pos_px_y_clean - meas_raft_pos_px_y_clean) / mean_px_per_m
# Find position errors relative to commands
x_err_clean = X.T - raft_x_mirror_coords
y_err_clean = Y.T - raft_y_mirror_coords

Mean pixels per meter: 4007.335485718933
Pixel per meter std. dev.: 38.355930858145406
Error in difference of two pixel coordinates: 54.243477617033946 pixels per meter of distance


In [54]:
f2, axs = plt.subplots(ncols=2, squeeze=False, figsize=(16,4))
title = dataset_name + ' Error Map'
plt.suptitle(title)
axs[0][0].set_aspect('equal')
im_x = axs[0][0].contourf(X.T, Y.T, x_err_clean)
cbar = plt.colorbar(im_x, ax=axs[0][0])
cbar.set_label('Error Magnitude (m)')
axs[0][0].set_title('X Error (m)')
axs[0][0].set_xlabel('Commanded x-Position (m)')
axs[0][0].set_ylabel('Commanded y-Position (m)')

print('Max X Error:', np.nanmax(x_err_clean), np.unravel_index(x_err_clean.argmax(), x_err_clean.shape))
print('Min X Error:', np.nanmin(x_err_clean))

axs[0][1].set_aspect('equal')
im_y = axs[0][1].contourf(X.T, Y.T, y_err_clean)
cbar = plt.colorbar(im_y, ax=axs[0][1])
cbar.set_label('Error Magnitude (m)')
axs[0][1].set_title('Y Error (m)')
axs[0][1].set_xlabel('Commanded x-Position (m)') 
axs[0][1].set_ylabel('Commanded y-Position (m)')

print('Max Y Error:', np.nanmax(y_err_clean), np.unravel_index(y_err_clean.argmax(), y_err_clean.shape))
print('Min Y Error:', np.nanmin(y_err_clean))

f2.savefig(os.path.join('output', dataset_name, title + '.png'), dpi=300)

Max X Error: 0.006049595688359 (6, 3)
Min X Error: -0.0028575581287055285
Max Y Error: 0.013471848583152729 (3, 0)
Min Y Error: 0.0006488160906246154


In [55]:
f3, axes = plt.subplots()
title = dataset_name + ' Error Magnitude'
plt.suptitle(title)
axes.set_aspect('equal')
err_mag = np.sqrt(x_err_clean ** 2. + y_err_clean ** 2.)
im = axes.contourf(X.T, Y.T, err_mag)
cbar = plt.colorbar(im, ax=axes)
cbar.set_label('Error Magnitude (m)')
axes.set_xlabel('Commanded x-Position (m)')
axes.set_ylabel('Commanded y-Position (m)')

print('Max error magnitude:', np.nanmax(err_mag))
print('Min error magnitude:', np.nanmin(err_mag))
print('Error magnitude mean:', np.nanmean(err_mag))
print('Error magnitude std:', np.nanstd(err_mag))

f3.savefig(os.path.join('output', dataset_name, title + '.png'), dpi=300)

Max error magnitude: 0.013472210608658209
Min error magnitude: 0.004409763578662251
Error magnitude mean: 0.007975948645726063
Error magnitude std: 0.0023636691976786093


In [56]:
f4, axes = plt.subplots()
title = dataset_name + ' Raft Angular Offset'
plt.suptitle(title)
axes.set_aspect('equal')
im = axes.contourf(X.T, Y.T, raft_ang_clean)
cbar = plt.colorbar(im, ax=axes)
cbar.set_label('Raft Angle (deg)')
axes.set_xlabel('Commanded x-Position (m)')
axes.set_ylabel('Commanded y-Position (m)')

print('Max angle:', np.nanmax(raft_ang_clean))
print('Min angle:', np.nanmin(raft_ang_clean))
print('Mean angle:', np.nanmean(raft_ang_clean))
print('Std angle:', np.nanstd(raft_ang_clean))

f4.savefig(os.path.join('output', dataset_name, title + '.png'), dpi=300)

Max angle: -3.4000000000000234
Min angle: -10.0
Mean angle: -7.089795918367357
Std angle: 1.8555207089137555


In [57]:
f5, axes = plt.subplots()
axes.set_facecolor('gainsboro')
title = dataset_name + ' Commanded vs. Actual Positions'
plt.suptitle(title)
axes.scatter(X.T, Y.T, facecolor='none', color='k', label='Commanded')
axes.scatter(raft_x_mirror_coords, raft_y_mirror_coords, c=err_mag, label='Measured')
# https://stackoverflow.com/a/64546653
axes.quiver(
    raft_x_mirror_coords, raft_y_mirror_coords,
    -x_err_clean, -y_err_clean,
    err_mag,
    pivot='tip',
    angles='xy',
    scale_units='xy',
    scale=1,
    headwidth=2.5,
    headlength=4,
    edgecolors='k',
    linewidth=.5,
    width=4e-3
)
axes.set_xticks(xs)
axes.set_yticks(ys)
axes.set_aspect('equal')
axes.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
axes.grid(True, color='k', linestyle='--')
axes.set_xlabel('x-Position (m)')
axes.set_ylabel('y-Position (m)')
f5.tight_layout()

f5.savefig(os.path.join('output', dataset_name, title + '.png'), dpi=300)

In [58]:
f6, axes = plt.subplots()
title = dataset_name + ' Error Residuals'
plt.suptitle(title)
f6.suptitle('Error Residuals')
axes.set_aspect('equal')
axes.grid(True, color='k', linestyle='--')
axes.scatter(x_err_clean, y_err_clean)
axes.set_xlabel('x-Position (m)')
axes.set_ylabel('y-Position (m)')

f6.savefig(os.path.join('output', dataset_name, title + '.png'), dpi=300)

In [46]:
plt.close('all')